In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import os
from os.path import dirname, join as pjoin
import scipy.io as sio
from networkx.algorithms import community 
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [5]:
class GraphMeasures:
    def __init__(self,threshold):
        self.threshold = threshold
        self.avg_node_connect = None
        self.adj_mat = None
        return
    
    def get_eigenvector_centrality(self,G):
        return networkx.eigenvector_centrality(G)
    
    def get_betweenness_centrality (self, G):
        return networkx.betweenness_centrality(G)
        
    def get_node_avg (self, G):
        return networkx.average_node_connectivity(G)
    
    def get_comm_central(self, G):
        return (networkx.community.girvan_newman(G))
                
    def get_adj_threshold(self, adj_mat):
        
        adj_mat[adj_mat <= self.threshold] = 0
        adj_mat[adj_mat > self.threshold] = 1
        return adj_mat
        
    def get_graphs(self, g_type, xp_num,fmri = False):
        base_name = "correlation/fmri/" if fmri else "correlation/"
        graph_list = []
        user = "sub-xp"
        for i in range(1,23):
            file = base_name+user+str(xp_num)+"0"+str(i)+"\\" + g_type + ".mat"
            try:
                mat_contents = sio.loadmat(file)
                adj_mat = np.array(mat_contents["rho"])
                adj_mat = self.get_adj_threshold(adj_mat)

                G = nx.convert_matrix.from_numpy_matrix(adj_mat)
                graph_list.append(G) 
            except FileNotFoundError: 
                continue
        return graph_list
    
    def get_subj_graphs(self, sub, run):
        base_names = ["correlation/fmri/sub-xp", "correlation/sub-xp"]
        graph_list = []

        files = [base_name+sub+"/" + run + ".mat" for base_name in base_names]
        for file in files:
            try:
                mat_contents = sio.loadmat(file)
                adj_mat = np.array(mat_contents["rho"])
                adj_mat = self.get_adj_threshold(adj_mat)

                G = nx.convert_matrix.from_numpy_matrix(adj_mat)
                graph_list.append(G) 
            except FileNotFoundError: 
                continue
        return graph_list

In [6]:
class GraphPlots:
    def __init__(self):
        return 
    def get_between_graph(self,graph_list, g):
        t = g.get_betweenness_centrality(graph_list[0])
        for graph in graph_list: 
            
            d = g.get_betweenness_centrality(graph)

            plt.bar(d.keys(), d.values(), color='g')
       
            plt.show()
            return 
   

    def get_deg_v_connectivity(self,graph_list, name):
        for graph in graph_list:
            
            val = nx.average_degree_connectivity(graph)
            x = val.keys()
            y = val.values()
            
            plt.scatter(x,y, color = 'g')
            plt.xlabel("degree")
            plt.ylabel("avg connectivity")
            plt.title("degree versus k-avg connectivity of "+name)
            plt.savefig("degree_v_connectivity_xp1_"+name+".png")
            plt.show()


    def get_similarity(self,graph_list):
        central = []
        
        for graph in graph_list:
            pr = nx.pagerank(graph)
            ranked = [[key, pr[key]] for key in pr.keys()]
            sorted_ranked = sorted(ranked, key=lambda item: item[1])

            # this makes ranked work. No idea why. Seems really strange.
            sorted_ranked = [sorted_ranked[i].append(i) for i in range(len(sorted_ranked))]
            
            central.append(ranked)
        
        matrix = np.zeros((len(central),len(central)))
        for i in range(len(central)):
            for j in range(i, len(central)):
                matrix[i,j] = sum([abs(central[i][x][2]-central[j][x][2]) for x in range(len(central[i]))])
                matrix[j,i] = matrix[i,j]
        
        return matrix
    
    
    def get_node_avg(self,graph_list, g, file_name):
        avg_con = []
        for graph in graph_list: 
            d = g.get_node_avg(graph)
            avg_con.append(d)

        plt.hist(avg_con, color='g')
        plt.savefig(file_name)
        plt.show()
        return 

    
    def graph_avg_and_var(self,graph_list, t, xp_num, fmri = False):
       
        file_name = "xp"+str(xp_num)+"_" + t
        
        graph_degree_list = []
        for graph in graph_list:
            graph_degree_list.append(list(dict(graph.degree).values()))

        avg_list = np.mean(graph_degree_list,axis = 0)
        varinance_list = np.std(graph_degree_list,axis = 0)
        

        n = 63
        
        plt.bar(np.arange(0, n),np.array(avg_list), color='blue', align='center', alpha = 0.5, label = "avg")
        plt.bar( np.arange(0, n),np.array(varinance_list) , color='red', align='center',alpha = 0.5, label = "var")
        plt.tight_layout()
        plt.title("average and varience of "+ file_name)
        plt.xlabel("node number")
        plt.ylabel("measure ")
        
        measure = 'fmri' if fmri else 'eeg'
        
        try:
            plt.savefig("data/pictures/" + measure +"/var_avg_"+file_name+".png")
        except:
            os.makedirs("data/pictures/" + measure + "/")
            plt.savefig("data/pictures/" + measure +"/var_avg_"+file_name+".png")

        
        plt.legend()

        plt.show()
    


 

In [18]:
def similarity():
    g = GraphMeasures(0.75)
    subs = ["101","102","103","104","105","106","107","108","109","110","201","202","203","204","205","206","207","210","211","213","216","217","218","219","220","221","222"]
    runs = ["eegNF","fmriNF" ,"eegfmriNF","motorloc", "MIpost", "MIpre","1dNF_run-01","1dNF_run-02","1dNF_run-03","1dNF_run-04","MIpost","MIpre","2dNF_run-01","2dNF_run-02","2dNF_run-03"]
    matrix = np.zeros((len(subs),len(runs)))

    g_plots = GraphPlots()
    
    for i in range(len(subs)):
        for j in range(len(runs)):
            print(subs[i],runs[j])

            graph_list = g.get_subj_graphs(subs[i],runs[j])
            if len(graph_list) < 2: 
                   matrix[i,j] = None
                   continue
#             breakpoint()
            matrix[i,j] = g_plots.get_similarity(graph_list)[0,1]

    
    return matrix

mat = similarity()

101 eegNF
101 fmriNF
101 eegfmriNF
101 motorloc
101 MIpost
101 MIpre
101 1dNF_run-01
101 1dNF_run-02
101 1dNF_run-03
101 1dNF_run-04
101 MIpost
101 MIpre
101 2dNF_run-01
101 2dNF_run-02
101 2dNF_run-03
102 eegNF
102 fmriNF
102 eegfmriNF
102 motorloc
102 MIpost
102 MIpre
102 1dNF_run-01
102 1dNF_run-02
102 1dNF_run-03
102 1dNF_run-04
102 MIpost
102 MIpre
102 2dNF_run-01
102 2dNF_run-02
102 2dNF_run-03
103 eegNF
103 fmriNF
103 eegfmriNF
103 motorloc
103 MIpost
103 MIpre
103 1dNF_run-01
103 1dNF_run-02
103 1dNF_run-03
103 1dNF_run-04
103 MIpost
103 MIpre
103 2dNF_run-01
103 2dNF_run-02
103 2dNF_run-03
104 eegNF
104 fmriNF
104 eegfmriNF
104 motorloc
104 MIpost
104 MIpre
104 1dNF_run-01
104 1dNF_run-02
104 1dNF_run-03
104 1dNF_run-04
104 MIpost
104 MIpre
104 2dNF_run-01
104 2dNF_run-02
104 2dNF_run-03
105 eegNF
105 fmriNF
105 eegfmriNF
105 motorloc
105 MIpost
105 MIpre
105 1dNF_run-01
105 1dNF_run-02
105 1dNF_run-03
105 1dNF_run-04
105 MIpost
105 MIpre
105 2dNF_run-01
105 2dNF_run-02
105 2d

In [ ]:
def main():
    g_plots = GraphPlots()
    
    # EEG XP1
    graph_t_xp1 = ["eegNF","fmriNF" ,"eegfmriNF","motorloc", "MIpost", "MIpre"]
    for t in graph_t_xp1: 
        g = GraphMeasures(0.75)
        graph_list = g.get_graphs(t,1)
        g_plots.graph_avg_and_var(graph_list, t, 1)
    
    # EEG XP2
    graph_t_xp2 = ["1dNF_run-01","1dNF_run-02","1dNF_run-03","1dNF_run-04","MIpost","MIpre","2dNF_run-01","2dNF_run-02","2dNF_run-03"]
    for t in graph_t_xp2: 
        g = GraphMeasures(0.75)
        graph_list = g.get_graphs(t,2)
        g_plots.graph_avg_and_var(graph_list, t, 2)

        
    #FMRI XP1
    for t in graph_t_xp1: 
        print(t, "FMRI XP 1")
        g = GraphMeasures(0.75)
        graph_list = g.get_graphs(t,1,fmri = True)
        g_plots.graph_avg_and_var(graph_list, t, 1, fmri=True)
    
    #FMRI XP2
    for t in graph_t_xp2: 
        print(t, "FMRI XP 1")
        g = GraphMeasures(0.75)
        graph_list = g.get_graphs(t,2,fmri = True)
        g_plots.graph_avg_and_var(graph_list, t, 2, fmri=True)

In [ ]:
# main()
centrality()


In [38]:
g_plots = GraphPlots()


# print(mat)

print(np.nanmean(mat), np.nanstd(mat))

val = []
for i in range(100):
    graph_list = [nx.generators.random_graphs.fast_gnp_random_graph(63,.1),nx.generators.random_graphs.fast_gnp_random_graph(63,.1)]
    val.append( g_plots.get_similarity(graph_list)[0,1])

print(np.nanmean(val), np.nanstd(val))


print("So, based on centrality of all nodes, the fmri and eeg graphs for the same person on the same run are no different than that of a random graph")


1280.1878453038673 127.02245882375891
1322.38 111.31493879978554
So, based on centrality of all nodes, the fmri and eeg graphs for the same person on the same run are no different than that of a random graph


In [ ]:
graph_t, i =["eegNF","fmriNF" ,"eegfmriNF"], 0
for j in graph_t: 
    i+=1
    g = graph_measures(0.75)
    graph_list = g.get_graphs(graph_t.index(j))
    
    avg_node = get_eigenvector_centrality(graph_list, g, (j+str(i)))

